In [1]:
import csv
import pandas as pd
import networkx as nx

In [2]:
data_file = '/global/D1/projects/umod/dipp/Misinformation_Dataset/Result/location_with_value/Location_with_only_country_code_cat_4_only_2.csv'
population_data_file = '/home/dipp/Github/Master-Thesis-dipp/Misinformation_Dataset_Analysis/Country_Data_Extraction/Result/Country_Data_cat_4_only_2.csv'

In [3]:
## Includes self Loops

country_data = {}
with open(data_file, newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    count = 0
    for row in reader:
        if(row['i'] != 'None' and row['j'] != 'None'):
            
            souce_loc = row['i'].replace("'","")
            destination_loc = row['j'].replace("'","")
            #country_data[count] = [row['i'], row['j'], float(row['count'])]
            #if(float(row['count']) > 999):
            country_data[count] = [souce_loc, destination_loc, float(row['count'])]
            count+=1

In [4]:
df = pd.DataFrame.from_dict(country_data, orient='index', columns=['Source','Destination','Number of Tweets'])

In [5]:
df

,Source,Destination,Number of Tweets
0,CH,CH,29.0
1,GB,GB,3340.0
2,US,US,40405.0
3,CN,CN,14.0
4,IL,IL,33.0
...,...,...,...
609,HT,US,1.0
610,GB,NP,1.0
611,CA,AR,1.0
612,TZ,US,1.0


## Creating Graph using Networkx

In [6]:
G = nx.from_pandas_edgelist(df, source='Source', target='Destination', edge_attr='Number of Tweets', create_using=nx.DiGraph())

In [7]:
G.degree(weight='Number of Tweets')['US']
G.in_degree(weight='Number of Tweets')['US']
G.out_degree(weight='Number of Tweets')['US']

44217.0

In [8]:
country = []
deg = G.degree(weight='Number of Tweets')
deg_in = G.in_degree(weight='Number of Tweets')
deg_out = G.out_degree(weight='Number of Tweets')

In [9]:
df['All Degree of Source'] = ''
df['All Degree of Destination'] = ''
df['Vi - Out Degree of Source'] = ''
df['Vj - In Degree of Destination'] = ''
# df['Percentage of Incoming'] = ''
# df['Total No. of Tweets/Population'] = ''
df

,Source,Destination,Number of Tweets,All Degree of Source,All Degree of Destination,Vi - Out Degree of Source,Vj - In Degree of Destination
0,CH,CH,29.0,,,,
1,GB,GB,3340.0,,,,
2,US,US,40405.0,,,,
3,CN,CN,14.0,,,,
4,IL,IL,33.0,,,,
...,...,...,...,...,...,...,...
609,HT,US,1.0,,,,
610,GB,NP,1.0,,,,
611,CA,AR,1.0,,,,
612,TZ,US,1.0,,,,


In [10]:
for index_df, row in df.iterrows():
    i = row['Source']
    j = row['Destination']

    for country_temp, deg_all in deg:
        if(country_temp == i):
            df.at[index_df, 'All Degree of Source'] = deg_all
        if(country_temp == j):
            df.at[index_df, 'All Degree of Destination'] = deg_all
    
    # Vj is in degree of destination

    for country_temp_in, deg_j in deg_in:
        if(country_temp_in == j):
            df.at[index_df, 'Vj - In Degree of Destination'] = deg_j
        # else:
        #     df.at[index_df, 'Vj - In Degree of Destination'] = 0
        

    # Vi is out degree of source

    for country_temp_out, deg_i in deg_out:
        if(country_temp_out == i):
            df.at[index_df, 'Vi - Out Degree of Source'] = deg_i
        # else:
        #     df.at[index_df, 'Vi - Out Degree of Source'] = 0    


In [11]:
df

,Source,Destination,Number of Tweets,All Degree of Source,All Degree of Destination,Vi - Out Degree of Source,Vj - In Degree of Destination
0,CH,CH,29.0,158.0,158.0,73.0,85.0
1,GB,GB,3340.0,9408.0,9408.0,4658.0,4750.0
2,US,US,40405.0,87071.0,87071.0,44217.0,42854.0
3,CN,CN,14.0,74.0,74.0,39.0,35.0
4,IL,IL,33.0,116.0,116.0,60.0,56.0
...,...,...,...,...,...,...,...
609,HT,US,1.0,1.0,87071.0,1.0,42854.0
610,GB,NP,1.0,9408.0,6.0,4658.0,4.0
611,CA,AR,1.0,4545.0,9.0,2013.0,7.0
612,TZ,US,1.0,3.0,87071.0,2.0,42854.0


In [12]:
df['Percentage of Incoming at Destination (Total Tweet / Vj)'] = ''
df['Percentage of Outgoing From Source (Total Tweet / Vi)'] = ''
df

,Source,Destination,Number of Tweets,All Degree of Source,All Degree of Destination,Vi - Out Degree of Source,Vj - In Degree of Destination,Percentage of Incoming at Destination (Total Tweet / Vj),Percentage of Outgoing From Source (Total Tweet / Vi)
0,CH,CH,29.0,158.0,158.0,73.0,85.0,,
1,GB,GB,3340.0,9408.0,9408.0,4658.0,4750.0,,
2,US,US,40405.0,87071.0,87071.0,44217.0,42854.0,,
3,CN,CN,14.0,74.0,74.0,39.0,35.0,,
4,IL,IL,33.0,116.0,116.0,60.0,56.0,,
...,...,...,...,...,...,...,...,...,...
609,HT,US,1.0,1.0,87071.0,1.0,42854.0,,
610,GB,NP,1.0,9408.0,6.0,4658.0,4.0,,
611,CA,AR,1.0,4545.0,9.0,2013.0,7.0,,
612,TZ,US,1.0,3.0,87071.0,2.0,42854.0,,


In [13]:
for index_df, row in df.iterrows():
    source = row['Source']
    destination = row['Destination']
    No_of_Tweets = row['Number of Tweets']
    vi = row['Vi - Out Degree of Source']
    vj = row['Vj - In Degree of Destination']

    per_in = float(No_of_Tweets / vj) * 100
    per_out = float(No_of_Tweets / vi) * 100
    df.at[index_df, 'Percentage of Incoming at Destination (Total Tweet / Vj)'] = per_in
    df.at[index_df, 'Percentage of Outgoing From Source (Total Tweet / Vi)'] = per_out

    


In [14]:
df

,Source,Destination,Number of Tweets,All Degree of Source,All Degree of Destination,Vi - Out Degree of Source,Vj - In Degree of Destination,Percentage of Incoming at Destination (Total Tweet / Vj),Percentage of Outgoing From Source (Total Tweet / Vi)
0,CH,CH,29.0,158.0,158.0,73.0,85.0,34.117647,39.726027
1,GB,GB,3340.0,9408.0,9408.0,4658.0,4750.0,70.315789,71.704594
2,US,US,40405.0,87071.0,87071.0,44217.0,42854.0,94.285248,91.378881
3,CN,CN,14.0,74.0,74.0,39.0,35.0,40.0,35.897436
4,IL,IL,33.0,116.0,116.0,60.0,56.0,58.928571,55.0
...,...,...,...,...,...,...,...,...,...
609,HT,US,1.0,1.0,87071.0,1.0,42854.0,0.002334,100.0
610,GB,NP,1.0,9408.0,6.0,4658.0,4.0,25.0,0.021468
611,CA,AR,1.0,4545.0,9.0,2013.0,7.0,14.285714,0.049677
612,TZ,US,1.0,3.0,87071.0,2.0,42854.0,0.002334,50.0


## Extensions

### Tweets per population and 2 versions of Cij 

In [15]:
population_data = {}

with open(population_data_file, newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    
    for row in reader:
        # print(row['Country Code'])
        # print(row['Population'])
        if(row['Population']!=''):
            population_data[row['Country Code']] = int(row['Population'])


In [16]:
population_data['AI'] = 15753
population_data['AQ'] = 3000
population_data['AX'] = 30129
population_data['BQ'] = 27726
population_data['CX'] = 1692
population_data['EH'] = 612000
population_data['FK'] = 3662
population_data['GF'] = 301099
population_data['GG'] = 63950
population_data['GS'] = 1
population_data['IO'] = 3000
population_data['JE'] = 103267
population_data['PN'] = 47
population_data['RE'] = 873102
population_data['SH'] = 4439
population_data['TF'] = 300
population_data['TW'] = 23894394
population_data['UM'] = 300
population_data['VA'] = 453
population_data['YT'] = 310022

In [17]:
df['Tweets Per Population'] = ''
df['Cij (All Degrees)'] = ''
df['Cij'] = ''

In [18]:
df

,Source,Destination,Number of Tweets,All Degree of Source,All Degree of Destination,Vi - Out Degree of Source,Vj - In Degree of Destination,Percentage of Incoming at Destination (Total Tweet / Vj),Percentage of Outgoing From Source (Total Tweet / Vi),Tweets Per Population,Cij (All Degrees),Cij
0,CH,CH,29.0,158.0,158.0,73.0,85.0,34.117647,39.726027,,,
1,GB,GB,3340.0,9408.0,9408.0,4658.0,4750.0,70.315789,71.704594,,,
2,US,US,40405.0,87071.0,87071.0,44217.0,42854.0,94.285248,91.378881,,,
3,CN,CN,14.0,74.0,74.0,39.0,35.0,40.0,35.897436,,,
4,IL,IL,33.0,116.0,116.0,60.0,56.0,58.928571,55.0,,,
...,...,...,...,...,...,...,...,...,...,...,...,...
609,HT,US,1.0,1.0,87071.0,1.0,42854.0,0.002334,100.0,,,
610,GB,NP,1.0,9408.0,6.0,4658.0,4.0,25.0,0.021468,,,
611,CA,AR,1.0,4545.0,9.0,2013.0,7.0,14.285714,0.049677,,,
612,TZ,US,1.0,3.0,87071.0,2.0,42854.0,0.002334,50.0,,,


In [19]:
import pypopulation

In [20]:
temp = 0
None_Type_Countries = []
for index_df, row in df.iterrows():
    source = row['Source']
    destination = row['Destination']
    No_of_Tweets = row['Number of Tweets']


    # source_poplation = population_data.get(source)
    # dest_population = population_data.get(destination)


    source_poplation = pypopulation.get_population(source)
    dest_population = pypopulation.get_population(destination)

    if(source_poplation == None):
        source_poplation = population_data.get(source)
            

    if(dest_population == None):
        dest_population = population_data.get(destination)
            

    percentage = No_of_Tweets / (source_poplation + dest_population)

    df.at[index_df, 'Tweets Per Population'] = percentage

    

In [21]:
df

,Source,Destination,Number of Tweets,All Degree of Source,All Degree of Destination,Vi - Out Degree of Source,Vj - In Degree of Destination,Percentage of Incoming at Destination (Total Tweet / Vj),Percentage of Outgoing From Source (Total Tweet / Vi),Tweets Per Population,Cij (All Degrees),Cij
0,CH,CH,29.0,158.0,158.0,73.0,85.0,34.117647,39.726027,0.000002,,
1,GB,GB,3340.0,9408.0,9408.0,4658.0,4750.0,70.315789,71.704594,0.000025,,
2,US,US,40405.0,87071.0,87071.0,44217.0,42854.0,94.285248,91.378881,0.000062,,
3,CN,CN,14.0,74.0,74.0,39.0,35.0,40.0,35.897436,0.0,,
4,IL,IL,33.0,116.0,116.0,60.0,56.0,58.928571,55.0,0.000002,,
...,...,...,...,...,...,...,...,...,...,...,...,...
609,HT,US,1.0,1.0,87071.0,1.0,42854.0,0.002334,100.0,0.0,,
610,GB,NP,1.0,9408.0,6.0,4658.0,4.0,25.0,0.021468,0.0,,
611,CA,AR,1.0,4545.0,9.0,2013.0,7.0,14.285714,0.049677,0.0,,
612,TZ,US,1.0,3.0,87071.0,2.0,42854.0,0.002334,50.0,0.0,,


In [22]:
for index_df, row in df.iterrows():
    source = row['Source']
    destination = row['Destination']
    No_of_Tweets = row['Number of Tweets']
    all_degree_source = row['All Degree of Source']
    all_degree_dest = row['All Degree of Destination']
    vi = row['Vi - Out Degree of Source']
    vj = row['Vj - In Degree of Destination']


    cij_all = No_of_Tweets / (all_degree_source + all_degree_dest)
    cij = No_of_Tweets / (vi + vj)

    df.at[index_df, 'Cij (All Degrees)'] = cij_all
    df.at[index_df, 'Cij'] = cij
    

In [23]:
df

,Source,Destination,Number of Tweets,All Degree of Source,All Degree of Destination,Vi - Out Degree of Source,Vj - In Degree of Destination,Percentage of Incoming at Destination (Total Tweet / Vj),Percentage of Outgoing From Source (Total Tweet / Vi),Tweets Per Population,Cij (All Degrees),Cij
0,CH,CH,29.0,158.0,158.0,73.0,85.0,34.117647,39.726027,0.000002,0.091772,0.183544
1,GB,GB,3340.0,9408.0,9408.0,4658.0,4750.0,70.315789,71.704594,0.000025,0.177509,0.355017
2,US,US,40405.0,87071.0,87071.0,44217.0,42854.0,94.285248,91.378881,0.000062,0.232023,0.464047
3,CN,CN,14.0,74.0,74.0,39.0,35.0,40.0,35.897436,0.0,0.094595,0.189189
4,IL,IL,33.0,116.0,116.0,60.0,56.0,58.928571,55.0,0.000002,0.142241,0.284483
...,...,...,...,...,...,...,...,...,...,...,...,...
609,HT,US,1.0,1.0,87071.0,1.0,42854.0,0.002334,100.0,0.0,0.000011,0.000023
610,GB,NP,1.0,9408.0,6.0,4658.0,4.0,25.0,0.021468,0.0,0.000106,0.000215
611,CA,AR,1.0,4545.0,9.0,2013.0,7.0,14.285714,0.049677,0.0,0.00022,0.000495
612,TZ,US,1.0,3.0,87071.0,2.0,42854.0,0.002334,50.0,0.0,0.000011,0.000023


In [24]:
df.to_csv('/home/dipp/Github/Master-Thesis-dipp/Misinformation_Dataset_Analysis/Network_Analysis/Result/global_network_cat_4_only_2.csv', index=False)